Документация Spark  
https://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html#pyspark.sql.functions.regexp_extract

In [459]:
import re
import json
import datetime
import pandas as pd
from pyspark.sql.functions import col,length,split, udf, initcap, regexp_extract, to_date, date_format,date_add,lower
from sys import argv
from pyspark import SparkConf, SparkContext
from pyspark.sql import HiveContext, SQLContext, SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
from #### import vertica
from #### import get_spark_session
from pyspark.sql.window import Window

In [ ]:
# Выводим номерацию колонок 

def numering(df):
    n = 0
    for i in df.columns:
        if n < len(df.columns):
            print(f' сol_{n}'+' : '+f'{i}')
        n = n+1
numering(df_spark)

In [ ]:
# для удобства назначаем колонки номерами

col_0 = df_spark.columns[0]
col_1 = df_spark.columns[1]
col_2 = df_spark.columns[2]
col_3 = df_spark.columns[3]
col_4 = df_spark.columns[4]
col_5 = df_spark.columns[5]
col_6 = df_spark.columns[6]
col_7 = df_spark.columns[7]
col_8 = df_spark.columns[8]
col_9 = df_spark.columns[9]
col_10 = df_spark.columns[10]
col_11 = df_spark.columns[11]
col_12 = df_spark.columns[12]
col_13 = df_spark.columns[13]
col_14 = df_spark.columns[14]
col_15 = df_spark.columns[15]
col_16 = df_spark.columns[16]
# col_17 = df_spark.columns[17]
# col_18 = df_spark.columns[18]
# col_19 = df_spark.columns[19]

## ФУНКЦИИ

#### Для <font color="green">ВСЕХ</font> колонок во Frame

In [ ]:
# Для ВСЕХ колонок во Frame
# функция для проверки данных

def looking_for_anomalies_in_columns(df):
    n = 0
    for i in df.columns:
        if n < len(df.columns):
            print(f"Колонка: {df.columns[n]}")
            print(f' ')
            print(f'Кол-во значений: {df.select(df.columns[n]).count()}')
            print(f'Кол-во уникальных значений: {df.select(df.columns[n]).distinct().count()}')
            print(f'Кол-во значений NaN: {df.select(df.columns[n]).filter(col(df.columns[n]).rlike("^[nan]*$|^[Nan]*$|^[null]*$")).count()}')
            print(f'Кол-во значений ^ пробелы: {df.select(df.columns[n]).filter(col(df.columns[n]).rlike("^ ")).count()}')
            print(f'Кол-во значений уникальных ^ пробелы: {df.select(df.columns[n]).filter(col(df.columns[n]).rlike("^ ")).distinct().count()}')
            print(f'Кол-во значений пробелы $: {df.select(df.columns[n]).filter(col(df.columns[n]).rlike(" $")).count()}')
            print(f'Кол-во значений уникальных $ пробелы: {df.select(df.columns[n]).filter(col(df.columns[n]).rlike(" $")).distinct().count()}')
            print(f'Странный пробел который не пробел(читается регуляркой как буква) : {df.select(df.columns[n]).filter(col(df.columns[n]).rlike(" ")).count()}')
            print(f' ')
        n = n+1   
        
looking_for_anomalies_in_columns(df)

#### Для <font color="blue">ОДНОЙ</font> колонки во Frame

In [ ]:
# Для просмотра ОДНОЙ колонки
# функция для проверки данных

def looking_for_anomalies(df,column): 
    print(f'Кол-во значений: {df.select(column).count()}')
    print(f'Кол-во уникальных значений: {df.select(column).distinct().count()}')
    print(f'Кол-во значений NaN: {df.select(column).filter(col(column).rlike("^[nan]*$|^[Nan]*$|^[null]*$")).count()}')
    print(f'Кол-во значений с цифрами: {df.select(column).filter(col(column).rlike("[0-9]+")).count()}')
    print(f'Кол-во значений в начале пробелы: {df.select(column).filter(col(column).rlike("^ ")).count()}')
    print(f'Кол-во значений уникальных в начале пробелы: {df.select(column).filter(col(column).rlike("^ ")).distinct().count()}')
    print(f'Кол-во значений пробелы в конце: {df.select(column).filter(col(column).rlike(" $")).count()}')
    print(f'Кол-во значений уникальных в конце пробелы : {df.select(column).filter(col(column).rlike(" $")).distinct().count()}')
    print(f'Странный пробел который не пробел(читается регуляркой как буква) : {df.select(column).filter(col(column).rlike(" ")).count()}')
    print(f'Кол-во значений UNKNOWN: {df.select(column).filter(col(column).rlike("UNKNOWN")).count()}')
    
looking_for_anomalies(df,'col')

In [ ]:
# функция для проверки данных на наличие букв и цифр

def check_letter_and_number(df,columns):    
    print(f'Кол-во уникальных значений: {df.select(columns).distinct().count()}')
    print(f'Кол-во значений NaN: {df.select(columns).filter(col(columns).rlike("^[nan]*$|^[Nan]*$")).count()}')
    print(f'Кол-во с буквами : {df.select(columns).filter(col(columns).rlike("[A-Za-zА-Яа-яЁё]")).count()}')
    print(f'Кол-во уникальных с буквами : {df.select(columns).filter(col(columns).rlike("[A-Za-zА-Яа-яЁё]")).distinct().count()}')
    print(f'Кол-во с цифрами : {df.select(columns).filter(col(columns).rlike("[0-9]")).count()}')
    print(f'Кол-во уникальных с цифрами : {df.select(columns).filter(col(columns).rlike("[0-9]")).distinct().count()}')
#     print(f'Кол-во уникальных символы: {df.select(columns).filter(col(columns).rlike("^[^0-9][^A-Za-zА-Яа-яЁё]$")).distinct().count()}')
#     print(f'Кол-во значений пробелы $: {df.select(columns).filter(col(columns).rlike(" $")).count()}')
#     print(f'Кол-во значений уникальных ^ пробелы: {df.select(columns).filter(col(columns).rlike(" $")).distinct().count()}')

check_letter_and_number(df,col_4)

In [ ]:
# функция для простой чистки данных

def clean_just(df,column):
    df = df.withColumn(column, F.regexp_replace(column, ' ',' '))\
        .withColumn(column, F.regexp_replace(column, ' {1,} ',' '))\
        .withColumn(column, F.regexp_replace(column, '^ ',''))\
        .withColumn(column, F.regexp_replace(column, ' $',''))\
        .withColumn(column, F.when(col(column).rlike("^[nan]*$|^[Nan]*$|^[null]*$"), 'UNKNOWN').otherwise(col(column)))

    return df

clean_just(df,'col')  #.select('col').distinct().show(100,False)

# ban

Где лежит черный список доменов (обновляется часто)

In [ ]:
# загружаем файл txt в DataFrame
df_txt = pd.read_csv("domens.txt", sep="\t", header=None)
# оборачиваем DataFrame в Spark DataFrame
df_domens = spark_session.createDataFrame(df_txt.astype(str))
# переименовываем колонку
df_domens = df_domens.withColumnRenamed('0','domens_1')

# Домены департаментов

In [ ]:
# загружаем список рабочих доменов
df_domen_job_orig = pd.read_excel("Domains2022.xlsx", engine='openpyxl')

# выводим общую информацию
df_domen_job_orig.info()

# выводим не обрезано 100 строчек
# pd.options.display.max_rows = 100

In [ ]:
# выводим первые 4 значения
df_domen_job_orig.head(4)

In [ ]:
# удаляем лишние строчки
df_domen_job_orig = df_domen_job_orig.drop(labels = [0,15],axis = 0)

# создаем новый DataFrame pandas чтобы добавить почты mosgorzdrav.ru','mosmedzdrav.ru
moszdrav = pd.DataFrame({'Name':['Неболей','Спорт'],
                          'DomainName':['nebolei.ru','Sport.ru'],
                        'DomainType':['Authoritative','Authoritative']})
#  Добавляем новые почты
df_domen_job_orig = df_domen_job_orig.append(moszdrav, ignore_index = True)

# оборачиваем DataFrame в Spark DataFrame
df_domen_job = spark_session.createDataFrame(df_domen_job_orig.astype(str)) 

# удаляем не нужную колонку
df_domen_job = df_domen_job.drop('DomainType')
# df_domen_job.show(2,False)

# ФИО

In [ ]:
# убираем дубли в строке с ФИО

# from pyspark.sql.functions import udf
# remove = udf(lambda row: (sorted(set(row.split(' ')), key=row.index)), returnType=StringType())
# df_join = udf(lambda row: (" ".join(row)), returnType=StringType())
# df_spark.withColumn(col_4, remove(col_4)).select(col_4)\
#     .withColumn(col_4, df_join(col_4)).select(col_4)\
#     .filter(col(col_4).rlike('Барсков')).show(5,False)

In [ ]:
# функция проверки ФИО

def check_fio(df,column): 
    sign = df.select(column).filter(col(column).rlike("._​\'A-Za-z!@$%^&*()+=;:\,/?\|\`~\[\]\}\{^\'\"<>")).count()
    
    print(f'Кол-во значений: {df.select(column).count()}')
    print(f'Кол-во уникальных значений: {df.select(column).distinct().count()}')
    print(f'Кол-во значений NaN: {df.select(column).filter(col(column).rlike("^[nan]*$")).count()}')
    print(f'Кол-во значений в начале пробелы: {df.select(column).filter(col(column).rlike("^ ")).count()}')
    print(f'Кол-во значений пробелы в конце: {df.select(column).filter(col(column).rlike(" $")).count()}')
    print(f'Кол-во значений с русскими буквами: {df.select(column).filter(col(column).rlike("[А-Яа-яЁё]+")).count()}')
    print(f'Кол-во значений с цифрами: {df.select(column).filter(col(column).rlike("[0-9]+")).count()}')
    print(f'Кол-во значений с латиницей и знаками: {sign}')
    print(f'Странный пробел который не пробел : {df.select(column).filter(col(column).rlike(" ")).count()}')
    
check_fio(df,col_4)

In [ ]:
# данные со знаком "(" (как мы видим данные после скобки нам не нужны в данном случае) но также
# встречаются скобки среди фамилий(нужно понять где меньше потерь) тогда можно использовать удаление "("

# Обработка ФИО

from pyspark.sql.functions import udf, initcap, regexp_extract
remove = udf(lambda row: (sorted(set(row.split(' ')), key=row.index)), returnType=StringType())
df_join = udf(lambda row: (" ".join(row)), returnType=StringType())

def clean_fio(df,column):
    df = df.withColumn(column,split(df[column],'\(').getItem(0))
    df_fio = df.withColumn(column, F.regexp_replace(column, ' ',' '))\
        .withColumn(column, remove(column)).withColumn(column, df_join(column))\
        .withColumn(column, F.when(col(column).rlike("^[nan]*$|^[Nan]*$|^[null]*$"), '').otherwise(col(column)))
    
    df_fio = df_fio.withColumn(column, F.when(col(column).rlike("([0-9]+)"),F.regexp_extract(column,"(^[^\s]*\s[^\s]*\s[^\s]*)",1)).otherwise(col(column)))

# для выполнения чтения перезаписываем в паркет(иногда без перезаписи все ломалось)
#    df_fio.write.mode('overwrite').parquet('./df_parquet.parquet')
#    df_fio = spark_session.read.parquet('./df_parquet.parquet')
    
    df_fio = df_fio.withColumn(column,F.regexp_replace(column, '[0-9._​\'A-Za-z!@$#%^&*()+=;:\,/?\|\`~\[\]{}^\'\"<>]+',''))\
        .withColumn(column,F.regexp_replace(column, ' {1,}',' '))\
        .withColumn(column,F.regexp_replace(column, ' [А-Яа-яЁё]{1} ',' '))\
        .withColumn(column,F.regexp_replace(column, ' Отделение| отделение| Дмитровское|совм|осн|оциф|оц|ФИО|пользователь|Временный| Внешсовм| К-з| Учитель| Гдзс| Ст| Лицей| Воспитатель| Восп| Основ| Основной| Основное| Осн| Внешнее Сов-во| Отд Энергоресурсы| Основной Работник| Гимназия Гтро| Внешний Совм| Внешний Совместитель| Помощник Воспитателя| исп физ лицо| Исп Физ Лицо| Тхэквондо',''))\
        .withColumn(column,initcap(column))\
        .withColumn(column,F.regexp_replace(column, ' $',''))\
        .withColumn(column,F.regexp_replace(column, 'вн$','вна'))\
        .withColumn(column,F.regexp_replace(column, 'ви$','вич'))\
        .withColumn(column,F.regexp_replace(column, 'ова$','овна'))\
        .withColumn(column,F.regexp_replace(column, ' - |- | -','-'))\
        .withColumn(column,F.regexp_replace(column, ' {1,}',' '))\
        .withColumn(column,F.regexp_replace(column, '^ ',''))\
        .withColumn(column,F.regexp_replace(column, '[ ]$',''))\
        .withColumn(column, F.when(length(column) <= 5, 'UNKNOWN').otherwise(col(column)))
        
    return df_fio
    
clean_fio(df,col_4)

In [ ]:
# КОММЕНТАРИИ К КОДУ ВЫШЕ

# Обработка ФИО

# чтобы убрать дубли в строке
# разбиваем строку на слова и сортируем по индексу(слова возвращаем на свои места)
remove = udf(lambda row: (sorted(set(row.split(' ')), key=row.index)), returnType=StringType())
# собираем слова в строчку
df_join = udf(lambda row: (" ".join(row)), returnType=StringType())
.withColumn(column, remove(column)).withColumn(column, df_join(column))\

def clean_fio(df,column):
    # Откину все после знака '('
.withColumn(column,split(df[column],'\(').getItem(0))

# удаляем странный пробел который не пробел(читается регуляркой как буква)
.withColumn(column, F.regexp_replace(column, ' ',' '))\

# убираем дубли в строке используя переменные выше
.withColumn(column, remove(column)).withColumn(column, df_join(column))\

# заменяем nan на знак '' 
.withColumn(column, F.when(col(column).rlike("^[nan]*$|^[Nan]*$|^[null]*$"), '').otherwise(col(column)))\

# выбираем строчки с цифрами и забираем только первые три слова
df_parquet = df.withColumn(column, F.when(col(column).rlike("([0-9]+)"),F.regexp_extract(column,"(^[^\s]*\s[^\s]*\s[^\s]*)",1))\
                .otherwise(col(column)))\

# для выполнения чтения перезаписываем в паркет(выше операция после выполнения без перезаписи все ломалось)
df_parquet.write.mode('overwrite').parquet('./df_parquet.parquet')
df = spark_session.read.parquet('./df_parquet.parquet')

# Убрать все цифры,латиницу и знаки
.withColumn(column,F.regexp_replace(column, '[0-9._​\'A-Za-z!@#$%^&*()+=;:\,/?\|\`~\[\]{}^\'\"<>]+',''))\

# Убрать множественные пробелы
.withColumn(column,F.regexp_replace(column, ' {1,}',' '))\

# В единичном виде знаки и слова убрать
.withColumn(column,F.regexp_replace(column, ' [А-Яа-яЁё]{1} ',' '))\

# Убрать лишние слова(то что нашел в таблицах)
.withColumn(column,F.regexp_replace(column, ' Отделение| отделение| Дмитровское|совм|осн|оциф|оц|ФИО|пользователь|Временный| Внешсовм| К-з| Учитель| Гдзс| Ст| Лицей| Воспитатель| Восп| Основ| Основной| Основное| Осн| Внешнее Сов-во| Отд Энергоресурсы| Основной Работник| Гимназия Гтро| Внешний Совм| Внешний Совместитель| Помощник Воспитателя| исп физ лицо| Исп Физ Лицо| Тхэквондо',''))\

# Выбрать регистр для слов(Первая буква в верхнем регистре)
.withColumn(column,initcap(column))\

# Убрать пробелы, поменять окончания вн и ви на вна и вич
.withColumn(column,F.regexp_replace(column, ' $',''))\
.withColumn(column,F.regexp_replace(column, 'вн$','вна'))\
.withColumn(column,F.regexp_replace(column, 'ви$','вич'))\
.withColumn(column,F.regexp_replace(column, 'ова$','овна'))\

# Убрать пробелы в начале и в конце
.withColumn(column,F.regexp_replace(column, '^[ ]',''))\
.withColumn(column,F.regexp_replace(column, '[ ]$',''))\

# Убрать лишние пробелы около тире
.withColumn(column,F.regexp_replace(column, ' - ','-'))\

# заменяем все что меньше или равно 5-ти кол-во символов на UNKNOWN
.withColumn(column, F.when(length(column) <= 5, 'UNKNOWN').otherwise(col(column)))

# 'СНИЛС'

In [ ]:
# Функция для очистки СНИЛС

def clean_snils(df,column):
    df_correct = df.withColumn(column, F.regexp_replace(column, ' ',' '))\
        .withColumn(column,F.regexp_replace(column, '^ {0,}',''))\
        .withColumn(column,F.regexp_replace(column, ' {0,}$',''))\
        .withColumn(column, F.regexp_replace(column, '^[!@#$%^&*()+=;:,-/?\|\` ~\[\]{}^\'\"<>]+$',''))\
        .withColumn(column, F.regexp_replace(column, '[- ]+',''))\
        .withColumn(column,F.regexp_replace(column, '^[nan]*$|^[Nan]*$|^[null]*$','UNKNOWN'))\
        .withColumn(column, F.when(length(column) < 11, 'UNKNOWN').when(length(column) > 11, 'UNKNOWN').otherwise(col(column)))
    
    return df_correct

clean_snils(df_spark,col_5)

# 'ДР'

In [ ]:
# нужно смотреть формат написания 
# 07.11.1983
# 1983-11-07
# 07-11-1983
# 1993-06-10 00:00:00

# df_spark.withColumn(col_6,F.regexp_replace(col_6, '^[nan]*$|^[Nan]*$|^[null]*$','UNKNOWN'))

In [ ]:
# 1-ое решение через date_format и to_date

# Функция для ДР и Даты выдачи паспорта
# приводим к формату dd.MM.yyyy(07.01.1996)
# from pyspark.sql.functions import to_date, date_format,date_add

def format_dr(df,column):
        if df.select(column).filter(col(column).rlike('\d\d\d\d-\d\d\-\d\d')).count() > 1000:
            df = df.withColumn(column, date_format(col(column),"dd.MM.yyyy"))
        elif df.select(column).filter(col(column).rlike('\d\d\d\d.\d\d\.\d\d')).count() > 1000:
            df = df.withColumn(column, date_format(col(column),"dd.MM.yyyy"))
        elif df.select(column).filter(col(column).rlike('\d{2}\-\d{2}\-\d{4}')).count() > 1000:
            df = df.withColumn(column, date_format(col(column),"dd.MM.yyyy"))
        elif df.select(column).filter(col(column).rlike('\d{2}\.\d{2}\.\d{4}')).count() > 1000:
            df = df.withColumn(column, to_date(col(column),"dd.MM.yyyy"))\
                .withColumn(column, date_format(col(column),"dd.MM.yyyy"))
        df = df.withColumn(column, F.when(col(column).isNull(), 'UNKNOWN').otherwise(col(column)))
        return df
        
format_dr(df_spark,col_6) #.select(col_6).show(3,False)

In [ ]:
# 2-ое решение через регулярки

# Функция для ДР и Даты выдачи паспорта
# приводим к формату dd.MM.yyyy(07.01.1996)

def format_dr(df,column):
        df = df.withColumn(column,F.regexp_replace(column, '^[nan]*$|^[Nan]*$|^[null]*$','UNKNOWN'))\
            .withColumn(column, F.regexp_replace(column, ' +',''))
        
        if df.select(column).filter(col(column).rlike('\d\d\d\d-\d\d\-\d\d')).count() > 1000:
            df = df.withColumn(column, F.regexp_replace(column, '(\d{4})\-(\d{2})\-(\d{2})','$3.$2.$1'))
        elif df.select(column).filter(col(column).rlike('\d{4}\.\d{2}\.\d{2}')).count() > 1000:
            df = df.withColumn(column, F.regexp_replace(column, '(\d{4})\.(\d{2})\.(\d{2})','$3.$2.$1'))
        elif df.select(column).filter(col(column).rlike('\d{2}\-\d{2}\-\d{4}')).count() > 1000:
            df = df.withColumn(column, F.regexp_replace(column, '[-]','.'))
                
        
        df = df.withColumn(column, col(column).substr(0, 10))
        
        return df
    
format_dr(df_spark,col_6) #.select(col_6).filter(col(col_6).rlike('UNKNOWN')).show(3,False)

# 'ТипДУЛ'

In [ ]:
# ТипДУЛ (долго выполняется обработка(больше 30 мин) c 1 млн. строк)
# Вариант разбить на несколько функции или использовать persist()
# можно промежуточно перезаписывать в паркет и читать их сразу.

def tipdul(df,column):
    df.withColumn(column, F.regexp_replace(column, ' ',' '))\
        .withColumn(column, F.regexp_replace(column, ' {1,}',' '))\
        .withColumn(column, F.regexp_replace(column, '^ | $',''))\
        .withColumn(column, F.regexp_replace(column, 'Форма представления сведений','UNKNOWN'))\
        .withColumn(column, F.when(col(column).rlike("Паспорт СССР|паспорт СССР|Паспорт гражданина СССР"), 'Паспорт гражданина СССР').otherwise(col(column)))\
        .withColumn(column, F.when(col(column).rlike("ПАСПОРТ РОССИИ|Паспорт гражданина РФ|Паспорт гражданина России|^паспорт РФ$|^Паспорт РФ$|^паспорт гражданина РФ$|^Паспорт гражданина РФ$|^Паспорт гражданина Российской Федерации$|^паспорт гражданина Российской Федерации$"), 'Паспорт гражданина РФ').otherwise(col(column)))\
        .withColumn(column, F.when(col(column).rlike("^Заграничный паспорт гражданина Р|^Загранпаспорт гражданина Р"), 'Загранпаспорт гражданина РФ').otherwise(col(column)))\
        .withColumn(column, F.when(col(column).rlike("Туркменистана|Заграничный паспорт гражданина иностранного государства|Белоруси|Паспорт гражданина Азербайджана|Паспорт гражданина Армении|аспорт Гражданина Украины|аспорт гражданина Украины|Республики|^паспорт иностранного|^Паспорт иностранного|ностранный паспорт|^Паспорт гражданина иностранного"), 'Паспорт гражданина иностранного государства').otherwise(col(column)))\
        .withColumn(column, F.when(col(column).rlike("Документ иностранного гражданина"), 'Документ иностранного гражданина').otherwise(col(column)))\
        .withColumn(column, F.when(col(column).rlike("Паспорт моряка|^паспорт$|^Паспорт$"), 'Паспорт').otherwise(col(column)))\
        .withColumn(column, F.when(col(column).rlike("ВИД НА ЖИТЕЛЬ|ид на жител"), 'Вид на жительство').otherwise(col(column)))\
        .withColumn(column, F.when(col(column).rlike("Иные докум|Трудовая книжка|Свидетельство о перемене имени|Свидетельство о регистрации по месту пребывания|Справка об освобождении|Полис ДМС|Страховой полис обязательного медицинского страхования гражданина Российской Федерации|Свидетельство о расторжении брака|основное|Водительское удостоверение|Полис ОМС|Патент|Свидетельство о заключении брака"), 'Иные документы').otherwise(col(column)))\
        .withColumn(column, F.when(col(column).rlike("Военный билет|Временное удостоверение, выданное взамен военного билета"), 'Военный билет').otherwise(col(column)))\
        .withColumn(column, F.when(col(column).rlike("Разрешение на врем"), 'Разрешение на временное проживание в РФ').otherwise(col(column)))\
        .withColumn(column, F.when(col(column).rlike("Свидетельство о рождении"), 'Свидетельство о рождении').otherwise(col(column)))\
        .withColumn(column, F.when(col(column).rlike("Временное удостоверение"), 'Временное удостоверение').otherwise(col(column)))\
        .withColumn(column, F.when(length(column) < 4, 'UNKNOWN').otherwise(col(column)))
            
    return df

tipdul(df,col_9)

# 'Документ_Серия'

In [ ]:
# Функция чистки Документ_Серия

def clean_doc(df,column):
    df_correct = df.withColumn(column, F.regexp_replace(column, ' ',' '))\
        .withColumn(column,F.regexp_replace(column, '^ {0,}| {0,}$',''))\
        .withColumn(column, F.regexp_replace(column, '^[!@#$%^&*()+=;:,-/?\|\` ~\[\]{}^\'\"<>]+$',''))\
        .withColumn(column, F.regexp_replace(column, '[- ]+',''))\
        .withColumn(column,F.regexp_replace(column, '^[nan]*$|^[Nan]*$|^[null]*$','UNKNOWN'))
    
    return df_correct

clean_snils(df_spark,col_10)

# <font color='green'>Phones</font>  

In [ ]:
# Функция для проверки ошибок в данных

def check_phones(df,column):
    print(f'Кол-во значений NaN: {df.select(column).filter(col(column).rlike("^[nan]*$|^[Nan]*$")).count()}')
    print(f'Кол-во значений с цифрами: {df.select(column).filter(col(column).rlike("[0-9]+")).count()}')
    print(f'Кол-во уникальных значений с цифрами: {df.select(column).filter(col(column).rlike("[0-9]+")).distinct().count()}')
    print(f'Кол-во тел. не соответствуют ^+ : {df.select(column).filter(col(column).rlike("^[^+]")).count()}')
    print(f'Кол-во уникальных не соответствуют ^+ : {df.select(column).filter(col(column).rlike("^[^+]")).distinct().count()}')
    print(f'Кол-во номеров с буквами : {df.select(column).filter(col(column).rlike("[A-Za-zА-Яа-яЁёёЁ]")).count()}')
    print(f'Кол-во уникальных номеров с буквами : {df.select(column).filter(col(column).rlike("[A-Za-zА-Яа-яЁёёЁ]")).distinct().count()}')
    print(f'Кол-во номеров с символами : {df.select(column).filter(col(column).rlike("^[^+]*$")).count()}')
    print(f'Кол-во уникальных с символами : {df.select(column).filter(col(column).rlike("^[^+]*$")).distinct().count()}')
    print(f'Странный пробел который не пробел : {df.select(column).filter(col(column).rlike(" ")).count()}')
    
check_phones(df_spark,col_13)

In [ ]:
# приводим все телефоны к единому формату +74957773388
# в строке попадаются "сестра,мама" владалец телефона(выносим в отдельную колонку 'Владелец_телефона')

def clean_mobile_number(df,column,newcolumn):
    # дублируем колонку
    df_mobile = df.withColumn(column, F.regexp_replace(column, ' ',' '))\
        .withColumn(newcolumn,col(column))\
        .withColumn(newcolumn,F.regexp_replace(newcolumn, '[0-9+)t,¶№&;?*/|` --.—\=:(][ -]*','9'))\
        .withColumn(newcolumn,F.regexp_replace(newcolumn, '[9]+',' '))\
        .withColumn(newcolumn,F.regexp_replace(newcolumn, '^ {1,}| {1,}$',''))\
        .withColumn(newcolumn, F.when(length(newcolumn) < 2, 'UNKNOWN')\
                    .when(length(newcolumn) > 14, 'UNKNOWN').otherwise(col(newcolumn)))\
        .withColumn(column,F.regexp_replace(column, '[A-Za-zА-Яа-яЁёёЁ]',''))\
        .withColumn(column,F.regexp_replace(column, '[¶№!@#$%^&*.\)\(\+=—;:,\-/?\|\` ~\[\]{}^\'\’\"<>]+',''))\
        .withColumn(column, F.when(length(column) < 2, 'UNKNOWN').otherwise(col(column)))\
        .withColumn(column,F.regexp_replace(column, '^[8]',''))\
        .withColumn(column,F.regexp_replace(column, '^[7]',''))\
        .withColumn(column, F.when(col(column).rlike("^[0123567]"), 'UNKNOWN').otherwise(col(column)))\
        .withColumn(column, F.when(length(column) <= 9, 'UNKNOWN').when(length(column) >= 11, 'UNKNOWN').otherwise(col(column)))\
        .withColumn(column,F.regexp_replace(column, '^','+7'))\
        .withColumn(column, F.when(col(column).rlike("[A-Za-zА-Яа-яЁёёЁ]"), 'UNKNOWN').otherwise(col(column)))
    
    return df_mobile

clean_mobile_number(df_spark,col_13,'Владелец_телефона')   

In [ ]:
# описание к функции выше

def clean_mobile_number(df,column,newcolumn):
    # удаляем пробел который не пробел(пробел видится как знак, два слова видятся как одно)
    .withColumn(column, F.regexp_replace(column, ' ',' '))\
    # дублируем колонку
    df_mobile = df.withColumn(newcolumn,col(column))\
    # превращаем все символы кроме букв превращаем в цифру 9
    .withColumn(newcolumn,F.regexp_replace(newcolumn, '[0-9+)t,¶№&;?*/|` --.—\=:(][ -]*','9'))\
    # все 9 превращаем в пробел
    .withColumn(newcolumn,F.regexp_replace(newcolumn, '[9]+',' '))\
    # убираем пробелы в начале и конце
    .withColumn(newcolumn,F.regexp_replace(newcolumn, '^ {1,}| {1,}$',''))\
    # затем все что меньше 2-ух знаков и больше 14-ти переводим в UNKNOWN
    .withColumn(newcolumn, F.when(length(newcolumn) < 2, 'UNKNOWN')\
                .when(length(newcolumn) > 14, 'UNKNOWN').otherwise(col(newcolumn)))\
    # удаляем все символы и букв кроме цифр
    .withColumn(column,F.regexp_replace(column, '[A-Za-zА-Яа-яЁёёЁ]',''))\
    # удаляем посторонние символы
    .withColumn(column,F.regexp_replace(column, '[¶№!@#$%^&*.\)\(\+=—;:,\-/?\|\` ~\[\]{}^\'\’\"<>]+',''))\
     # удаляем все что меньше 2-ух знаков
    .withColumn(column, F.when(length(column) < 2, 'UNKNOWN').otherwise(col(column)))\
    # удаляем цифру восемь в начале
    .withColumn(column,F.regexp_replace(column, '^[8]',''))\
    # удаляем цифру семь в начале
    .withColumn(column,F.regexp_replace(column, '^[7]',''))\
    # удаляем всё что не начинается на 4,8,9
    .withColumn(column, F.when(col(column).rlike("^[0123567]"), 'UNKNOWN').otherwise(col(column)))\
    # затем все что меньше 9-ти знаков и больше 11-ти переводим в UNKNOWN
    .withColumn(column, F.when(length(column) <= 9, 'UNKNOWN').when(length(column) >= 11, 'UNKNOWN').otherwise(col(column)))\
    # подставляем +7 в начале
    .withColumn(column,F.regexp_replace(column, '^','+7'))\
    # меняем любые совпадения букв на UNKNOWN
    .withColumn(column, F.when(col(column).rlike("[A-Za-zА-Яа-яЁёёЁ]+"), 'UNKNOWN').otherwise(col(column)))
    
    return df_mobile

clean_mobile_number(df_spark,col_13,'Владелец_телефона') 

# 'Номер_телефонаРабочий'  
попадается добавочный номер(выносим в отдельную колонку)

In [ ]:
# приводим все телефоны к единому формату +74957778811
# в строке попадаются "доб." разделяем по слову "доб."
# добавочный телефон выносим в отдельную колонку 'Внутреннийтел'

def clean_telefone_number(df,column,word,newcolumn):
    # Разделение колонки на две по слову
    df_correct = df.withColumn(column, F.regexp_replace(column, ' ',' '))\
        .withColumn(newcolumn,split(df[column],word).getItem(1))\
        .withColumn(newcolumn,F.regexp_replace(newcolumn, '[+)t,¶№&;?*/|` --.—\=:(][ -]*',''))\
        .withColumn(newcolumn,F.regexp_replace(newcolumn, '[A-Za-zА-Яа-яЁёёЁ]+',''))\
        .na.fill('UNKNOWN', newcolumn)\
        .withColumn(column,split(df[column],word).getItem(0))\
        .withColumn(column,F.regexp_replace(column, '[A-Za-zА-Яа-яЁёёЁ]',''))\
        .withColumn(column,F.regexp_replace(column, '[¶№!@#$%^&*.\)\(\+=—;:,\\-/?\|\` ~\[\]{}^\'\’\"<>]+',''))\
        .withColumn(column,F.regexp_replace(column, '^[8]',''))\
        .withColumn(column,F.regexp_replace(column, '^[7]',''))\
        .withColumn(column, F.when(col(column).rlike("^[0123567]"), 'UNKNOWN').otherwise(col(column)))\
        .withColumn(column, F.when(length(column) <= 9, 'UNKNOWN').when(length(column) >= 11, 'UNKNOWN').otherwise(col(column)))\
        .withColumn(column,F.regexp_replace(column, '^','+7'))\
        .withColumn(column, F.when(col(column).rlike("[A-Za-zА-Яа-яЁёёЁ]"), 'UNKNOWN').otherwise(col(column)))
    
    return df_correct

clean_telefone_number(df_spark,col_14,'доб','Внутреннийтел')

In [ ]:
# ОПИСАНИЕ к функции выше

def clean_telefone_number(df,column,word,newcolumn):
    # удаляем пробел который не пробел
    .withColumn(column, F.regexp_replace(column, ' ',' '))\
    # Разделение колонку на две по знаку 'доб',вторая колонка
    df_correct = df.withColumn(newcolumn,split(df[column],word).getItem(1))\
    # удаляем все лишнее (Символы) в второй колонке
    .withColumn(newcolumn,F.regexp_replace(newcolumn, '[¶№!@#$%^&*.\)\(\+=—;:,\\-/?\|\` ~\[\]{}^\'\’\"<>]+',''))\
    # удаляем все лишнее (буквы) в второй колонке
    .withColumn(newcolumn,F.regexp_replace(newcolumn, '[A-Za-zА-Яа-яЁёёЁ]+',''))\
    # заменяем в колонке "Внутреннийтел" значения null на UNKNOWN
    .na.fill('UNKNOWN', newcolumn)\
    # первая до знака 'доб'
    .withColumn(column,split(df[column],word).getItem(0))\
    # удаляем все лишнее (Символы) в первой колонке
    .withColumn(column,F.regexp_replace(column, '[¶№!@#$%^&*.\)\(\+=—;:,\\-/?\|\` ~\[\]{}^\'\’\"<>]+',''))\
    # удаляем все лишнее (буквы) в первой колонке
    .withColumn(column,F.regexp_replace(column, '[A-Za-zА-Яа-яЁёёЁ]',''))\
    # удаляем 8 в начале
    .withColumn(column,F.regexp_replace(column, '^[8]',''))\
    # удаляем 7 в начале
    .withColumn(column,F.regexp_replace(column, '^[7]',''))\
    # заменям значения начало которых начинается на 0,1,2,3,5,6,7 на значение 'UNKNOWN'
    .withColumn(column, F.when(col(column).rlike("^[0123567]"), 'UNKNOWN').otherwise(col(column)))\
    # затем все что меньше 9-ти знаков и больше 11-ти переводим в UNKNOWN
    .withColumn(column, F.when(length(column) <= 9, 'UNKNOWN').when(length(column) >= 11, 'UNKNOWN').otherwise(col(column)))\
    # подставляем +7 в начале
    .withColumn(column,F.regexp_replace(column, '^','+7'))\
    # меняем любые совпадения букв на UNKNOWN
    .withColumn(column, F.when(col(column).rlike("[A-Za-zА-Яа-яЁёёЁ]"), 'UNKNOWN').otherwise(col(column)))

    return df_correct

# 1-таблица, 2-колонку, 3-слово разделитель колонки, 4-название новой колонки(в которую записываем добав)
clean_telefone_number(df_spark,col_14,'доб','Внутреннийтел')

# Job Email

In [ ]:
## Список доменов по рабочей почте

df_domen_excel_orig = pd.read_excel("Domains2022.xlsx", engine='openpyxl')
df_domen_excel_orig.info()

# pd.options.display.max_rows = 100
# df_domen_excel_orig.head(100)

In [ ]:
df_domen_job_orig = df_domen_excel_orig.drop(labels = [0,15],axis = 0)

# создаем новый DataFrame pandas чтобы добавить почты mosgorzdrav.ru','mosmedzdrav.ru
moszdrav = pd.DataFrame({'Name':['Мосгорздрав','Мосмедздрав'],
                          'DomainName':['mosgorzdrav.ru','mosmedzdrav.ru'],
                        'DomainType':['Authoritative','Authoritative']})
#  Добавляем новые почты
df_domen_job_orig = df_domen_job_orig.append(moszdrav, ignore_index = True)

df_domen_job = spark_session.createDataFrame(df_domen_job_orig.astype(str)) 

df_domen_job = df_domen_job.drop('DomainType')
# df_domen_excel.show(82,False)

In [ ]:
# Функция для проверки ошибок в данных рабочей почты

def email_job_check(df,columns):
    # Кол-во c символами '.-><_' в начале
    sign_start = df.select(columns).filter(col(columns).rlike('^[!@#$%^&*()+=;:,/?\|\`~\[\]{}^\'\"<>]+')).count()
    # Кол-во c символами '.-><_' в конце
    sign_end = df.select(columns).filter(col(columns).rlike('[!@#$%^&*()+=;:,/?\|\`~\[\]{}^\'\"<>]+$')).count()
    # Кол-во строк с пробелами и знаком ¶
    probel = df.select(columns).filter(col(columns).rlike(' |¶|\\|#|\/|\"\'\`')).count()
    domen_mos = df.withColumn('domen',split(F.col(columns), '@')[1]).select('domen').filter(col('domen').rlike('(?i)mos')).count()
    domen_mos_distinct = df.withColumn('domen',split(F.col(columns), '@')[1]).select('domen').filter(col('domen').rlike('(?i)mos')).distinct().count()
    backslash = df.select(columns).filter(col(columns).rlike('<|>|\)|\(')).count()  
    
    print(f"Кол-во значений : {df.select(columns).count()}")
    print(f"Кол-во c 'nan' и 'null' : {df.select(columns).filter(col(columns).rlike('^[nan]*$|^[Nan]*$|^[null]*$|^[null]*$')).count()}")
    print(f"Кол-во строк с пробелами и знаком ¶ : {probel}")
    print(f"Кол-во строк без '@' : {df.select(columns).filter(col(columns).rlike('^[^@]+$')).count()}")
    print(f"Кол-во строк c двойными '@@' и больше : {df.select(columns).filter(col(columns).rlike('[@]{2,}+')).count()}")
    print(f"Кол-во строк c двумя '@' в строке и больше : {df.select(columns).filter(col(columns).rlike('[@]+[А-Яа-яЁёA-Za-z0-9_,.; :<>]+[@]+')).count()}")
    print(f"Кол-во строк почт без точек после '@' : {df.select(columns).filter(col(columns).rlike('@[^.]+$')).count()}")
    print(f"Кол-во только русские буквы и цифры с пробелом : {df.select(columns).filter(col(columns).rlike('^[А-Яа-яЁё 0-9]+$')).count()}")
    print(f"Кол-во c символами '.\/,->+*<_#' в начале : {sign_start}")
    print(f"Кол-во c символами '.\/,->+*<_#' в конце : {sign_end}")
    print(f"Кол-во c символами '<>)('  : {backslash}")
    print(f"Кол-во заканчивается на r или ry : {df.select(columns).filter(col(columns).rlike('(?i)[.]r$|(?i)[.]ry$')).count()}")
    print(f"Кол-во рабочих почт в домене есть 'mos'  : {domen_mos}")
    print(f"Кол-во рабочих почт в домене уникальных есть 'mos'  : {domen_mos_distinct}")
    print(f"Кол-во строк со словом 'e-mail:'  : {df.select(columns).filter(col(columns).rlike('e-mail:')).count()}")
    print(f'Странный пробел который не пробел : {df.select(column).filter(col(column).rlike(" ")).count()}')
    
    if df.select(columns).filter(col(columns).rlike('\)|\(')).distinct().count() >= 1:
        df.select(columns).filter(col(columns).rlike('\)|\(')).distinct().show(2,False)
    if df.select(columns).filter(col(columns).rlike('<|>')).distinct().count() >= 1:
        df.select(columns).filter(col(columns).rlike('<|>')).distinct().show(2,False)
    if df.select(columns).filter(col(columns).rlike('e-mail:')).distinct().count() >= 1:
        df.select(columns).filter(col(columns).rlike('e-mail:')).distinct().show(2,False)
    else:
        return
    
email_job_check(df_spark,col_15)
# df_spark.select(col_16).filter(col(col_16).rlike('e-mail:')).distinct().show(413,False)

In [ ]:
# Обработка рабочих почт

def clean_job_email(df,column,df_job):
    df_email = df.withColumn(column, F.regexp_replace(column, ' ',' '))\
        .withColumn(column, F.when(col(column).rlike("^[^@]*$"), 'UNKNOWN').otherwise(col(column)))\
        .withColumn(column, F.regexp_replace(column, ' |¶|\\|#|\/|\"\'\`',''))\
        .withColumn(column, F.when(col(column).rlike("@[^.]+$"), 'UNKNOWN').otherwise(col(column)))\
        .withColumn(column, F.when(col(column).rlike("[@]{2,}+"), 'UNKNOWN').otherwise(col(column)))\
        .withColumn(column, F.when(col(column).rlike("[@]+[А-Яа-яЁёA-Za-z0-9_,.; :<>]+[@]+"), 'UNKNOWN').otherwise(col(column)))\
        .withColumn(column, F.regexp_replace(column, 'e-mail:',''))\
        .withColumn(column, F.regexp_replace(column, '^[!@#$%^&*()+=;:,/?\|\` ~\[\]{}^\'\"<>]+',''))\
        .withColumn(column, F.regexp_replace(column, '[!@#$%^&*()+=;:,/?\|\` ~\[\]{}^\'\"<>]+$',''))\
        .withColumn(column, F.regexp_replace(column, '(?i)[.]r$|(?i)[.]ry$','.ru'))\
        .withColumn(column, F.regexp_replace(column, '(?i)moc.ru$','mos.ru'))
    
    df_email = df_email.withColumn('domen',split(F.col(column), '@')[1])\
        .withColumn('domen', F.regexp_replace('domen', '^[¶!@#$%^&*()+=;\- :.,/?\|\`~\[\]{}^\'\"<>]',''))\
        .withColumn('domen', F.regexp_replace('domen', '[А-Яа-яЁё¶!@#$%^&*()+=;\- :.,/?\|\`~\[\]{}^\'\"<>]+$',''))\
        .withColumn('domen', F.regexp_replace('domen', '(?i)[.]r$|(?i)[.]ry$|(?i)[.]u$','.ru'))\
        .withColumn('mos', F.when(col('domen').rlike('(?i)mos|(?i)russpass'), '#').otherwise('UNKNOWN'))\
        .withColumn(column, F.when(col('mos').rlike("[^#]"), 'UNKNOWN').otherwise(col(column)))\
        .drop('mos')
    
    df_email = df_email.join(df_job, df_email.domen==df_job.DomainName,'left')\
        .withColumn(column, F.when(col('DomainName').isNull(), 'UNKNOWN').otherwise(col(column)))\
        .drop('domen', 'DomainName','Name')
        
    return df_email

clean_job_email(df_spark,col_15,df_domen_job)

In [5]:
# Описание функции рабочей почты

def clean_job_email(df,column,df1):
    # Удаляем пробел который не пробел
    df_email = df.withColumn(column, F.regexp_replace(column, ' ',' '))\
        # Убираем почты без знака "@"
        .withColumn(column, F.when(col(column).rlike("^[^@]*$"), 'UNKNOWN').otherwise(col(column)))\
        # Убираем все пробелы и
        .withColumn(column, F.regexp_replace(column, ' |¶|\\|#|\/|\"\'\`',''))\
        # Убираем почты без точек после "@"
        .withColumn(column, F.when(col(column).rlike("@[^.]+$"), 'UNKNOWN').otherwise(col(column)))\
        # Убираем если двойная "@@" и больше
        .withColumn(column, F.when(col(column).rlike("[@]{2,}$"), 'UNKNOWN').otherwise(col(column)))\
        # Убираем если больше двух "@" 
        .withColumn(column, F.when(col(column).rlike("[@]+[А-Яа-яA-Za-z0-9_,.; :<>]+[@]+"), 'UNKNOWN').otherwise(col(column)))\
        # Убираем "e-mail:" 
        .withColumn(column, F.regexp_replace(column, 'e-mail:',''))
        # Убираем символы в начале почты(точки и запятые)
        .withColumn(column, F.regexp_replace(column, '^[!@#$%^&*()+=;:,/?\|\` ~\[\]{}^\'\"<>]+',''))\
         # Убираем символы в конце почты(точки и запятые)
        .withColumn(column, F.regexp_replace(column, '[!@#$%^&*()+=;:,/?\|\` ~\[\]{}^\'\"<>]+$',''))\
        # Добавляем к "r" + "u" = ru
        .withColumn(column, F.regexp_replace(column, '(?i)[.]r$|(?i)[.]ry$','.ru'))\
        # Заменяем moc.ru на mos.ru
        .withColumn(column, F.regexp_replace(column, '(?i)moc.ru$','mos.ru'))
        # создаем колонку только домен второго уровня для проверки в забанненых
    df_email = df_email.withColumn('domen',split(F.col(col_16), '@')[1])\
         # Убираем символы в начале домена (точки и запятые)
        .withColumn('domen', F.regexp_replace('domen', '^[¶!@#$%^&*()+=;:,/?\|\`~\[\]{}^\'\"<>]',''))\
        # Убираем символы в конце домена (точки и запятые)
        .withColumn('domen', F.regexp_replace('domen', '[¶!@#$%^&*()+=;:,/?\|\`~\[\]{}^\'\"<>]+$',''))\
        # Добавляем к "r" + "u" = ru
        .withColumn('domen', F.regexp_replace('domen', '(?i)[.]r$|(?i)[.]ry$','.ru'))\
        # (оставляем только раб домены)Выбираем домены с mos и назначаем их # остальные UNKNOWN
        .withColumn('mos', F.when(col('domen').rlike('(?i)mos'), '#').otherwise('UNKNOWN'))\
        # (удаляем все не раб домены)Выбираем почты с mos без # и заменяем в почте как UNKNOWN
        .withColumn(column, F.when(col('mos').rlike("[^#]"), 'UNKNOWN').otherwise(col(column)))\
         # Удаляем колонку mos
        .drop('mos')
        # Джойним список с доменами второго уровня которые забаннены
    df_email = df_email.join(df1, df_email.domen==df1.domens_1,'left')\
        # В колонке 'column' убираем почты если они есть в колонке забанных
        .withColumn(column, F.when(col('domens_1').isNotNull(), 'ban_SUDIR').otherwise(col(column)))\
        # Удаляем колонки которые добавили в текущей функции
        .drop('domen', 'domens_1')
    
    return df_email
clean_job_email(df_spark,col_16,df_domens)

# <font color='green'>Email Личный</font>  

## Загружаем файл TXT с забаннеными доменами второго уровня

In [ ]:
df_txt = pd.read_csv("domens.txt", sep="\t",header= None)
df_domens = spark_session.createDataFrame(df_txt.astype(str))
df_domens = df_domens.withColumnRenamed('0','domens_1')

In [1]:
# Заменяем Punycode на его расшифровку
import idna
print (idna.decode('xn----7sba5bazifhfz.xn--p1ai'))

# Punycode конвертер для доменов 2 уровня типа - 'xn----7sba5bazifhfz.xn--p1ai'
# на выходе получаем - 'спарта-кфкс.рф'  

спарта-кфкс.рф


https://www.reg.ru/web-tools/punycode?_gl=1*1s9u8xp*_ga*NDc2MTA4MzE1LjE2NDA4NjA0OTc.*_ga_N9GCQPR82H*MTY0MTM4MDM0MC40LjAuMTY0MTM4MDM0MC42MA..

(?i) - Использовать соответствие без учета регистра

Список часто используемых доменов для личной почты

rambler.ru  
rambler.ua  
ro.ru  
lenta.ru  
autorambler.ru  
myrambler.ru  
  
mail.ru  
list.ru  
bk.ru  
internet.ru  
inbox.ru  

gmail.com  
googlemail.com  

yandex.ru  
yandex.by  
yandex.ua  
yandex.kz  
yandex.com  
ya.ru  
narod.ru  

YAHOO  
rocketmail.com  
yahoo.com  
ymail.com  

MICROSOFT  
hotmail.com  
live.com  
outlook.com

https://developer.roman.grinyov.name/blog/92

In [ ]:
# заготовки по почтам
# находим русские буквы и удаляем буквы
# пример: на входе - "олдлод@mail.ru" 
#         на выходе - "@mail.ru"

# rambler.ru
.withColumn(column, F.regexp_replace(column, r'(?i)[А-Яа-яЁё]+(?=@myrambler.ru)|(?i)[А-Яа-яЁё]+(?=@autorambler.ru)|(?i)[А-Яа-яЁё]+(?=@lenta.ru)|(?i)[А-Яа-яЁё]+(?=@ro.ru)|(?i)[А-Яа-яЁё]+(?=@rambler.ua)|(?i)[А-Яа-яЁё]+(?=@rambler.ru)',''))\
# mail.ru
.withColumn(column, F.regexp_replace(column, r'(?i)[А-Яа-яЁё]+(?=@mail.ru)|(?i)[А-Яа-яЁё]+(?=@list.ru)|(?i)[А-Яа-яЁё]+(?=@bk.ru)|(?i)[А-Яа-яЁё]+(?=@internet.ru)|(?i)[А-Яа-яЁё]+(?=@inbox.ru)',''))\
# gmail.com
.withColumn(column, F.regexp_replace(column, r'(?i)[А-Яа-яЁё]+(?=@gmail.com)|(?i)[А-Яа-яЁё]+(?=@googlemail.com)',''))\
# yandex.ru
.withColumn(column, F.regexp_replace(column, r'(?i)[А-Яа-яЁё]+(?=@yandex.ru)|(?i)[А-Яа-яЁё]+(?=@yandex.by)|(?i)[А-Яа-яЁё]+(?=@yandex.ua)|(?i)[А-Яа-яЁё]+(?=@yandex.kz)|(?i)[А-Яа-яЁё]+(?=@yandex.com)|(?i)[А-Яа-яЁё]+(?=@ya.ru)|(?i)[А-Яа-яЁё]+(?=@narod.ru)',''))\
# yahoo.com
.withColumn(column, F.regexp_replace(column, r'(?i)[А-Яа-яЁё]+(?=@ymail.com)|(?i)[А-Яа-яЁё]+(?=@rocketmail.com)|(?i)[А-Яа-яЁё]+(?=@yahoo.com)',''))\
# MICROSOFT
.withColumn(column, F.regexp_replace(column, r'(?i)[А-Яа-яЁё]+(?=@hotmail.com)|(?i)[А-Яа-яЁё]+(?=@live.com)|(?i)[А-Яа-яЁё]+(?=@outlook.com)',''))\

In [ ]:
# заготовки по почтам
# находим русские буквы и удаляем буквы
# пример: на входе - "ДМШ имени М.Л. Таривердиева <dmsh37@mail.ru"   
#         на выходе - "..dmsh37@mail.ru"

# rambler.ru
.withColumn(column, F.regexp_replace(column, r'(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@myrambler.ru)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@autorambler.ru)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@lenta.ru)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@ro.ru)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@rambler.ua)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@rambler.ru)',''))\
# mail.ru
.withColumn(column, F.regexp_replace(column, r'(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@mail.ru)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+list.ru)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@bk.ru)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+internet.ru)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+inbox.ru)',''))\
# gmail.com
.withColumn(column, F.regexp_replace(column, r'(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@gmail.com)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@googlemail.com)',''))\
# yandex.ru
.withColumn(column, F.regexp_replace(column, r'(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@yandex.ru)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@yandex.by)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@yandex.ua)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@yandex.kz)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@yandex.com)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@ya.ru)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@narod.ru)',''))\
# yahoo.com
.withColumn(column, F.regexp_replace(column, r'(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@ymail.com)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@rocketmail.com)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@yahoo.com)',''))\
# MICROSOFT
.withColumn(column, F.regexp_replace(column, r'(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@hotmail.com)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@live.com)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@outlook.com)',''))\

In [ ]:
# заготовки по почтам
# находим русские буквы и эти почты(всю строку) заменяем "UNKNOWN"
# пример: на входе - "олдлод@mail.ru" 
#         на выходе - "UNKNOWN"

# rambler.ru
# F.when(col(column).rlike("(?i)[А-Яа-яЁё]@myrambler.ru|(?i)[А-Яа-яЁё]@autorambler.ru|(?i)[А-Яа-яЁё]@lenta.ru|(?i)[А-Яа-яЁё]@ro.ru|(?i)[А-Яа-яЁё]@rambler.ua|(?i)[А-Яа-яЁё]@rambler.ru"), 'UNKNOWN').otherwise(col(column)))\
# mail.ru
# F.when(col(column).rlike("(?i)[А-Яа-яЁё]@mail.ru|(?i)[А-Яа-яЁё]@list.ru|(?i)[А-Яа-яЁё]@bk.ru|(?i)[А-Яа-яЁё]@internet.ru|(?i)[А-Яа-яЁё]@inbox.ru"), 'UNKNOWN').otherwise(col(column)))\
# gmail.com
# F.when(col(column).rlike("(?i)[А-Яа-яЁё]@gmail.com|(?i)[А-Яа-яЁё]@googlemail.com"), 'UNKNOWN').otherwise(col(column)))\
# yandex.ru
# F.when(col(column).rlike("(?i)[А-Яа-яЁё]@yandex.ru|(?i)[А-Яа-яЁё]@yandex.by|(?i)[А-Яа-яЁё]@yandex.ua|(?i)[А-Яа-яЁё]@yandex.kz|(?i)[А-Яа-яЁё]@yandex.com|(?i)[А-Яа-яЁё]@ya.ru|(?i)[А-Яа-яЁё]@narod.ru"), 'UNKNOWN').otherwise(col(column)))\
# yahoo.com
#F.when(col(column).rlike("(?i)[А-Яа-яЁё]@ymail.com|(?i)[А-Яа-яЁё]@rocketmail.com|(?i)[А-Яа-яЁё]@yahoo.com"), 'UNKNOWN').otherwise(col(column)))\
# MICROSOFT
# F.when(col(column).rlike("(?i)[А-Яа-яЁё]@hotmail.com|(?i)[А-Яа-яЁё]@live.com|(?i)[А-Яа-яЁё]@outlook.com"), 'UNKNOWN').otherwise(col(column)))\

Есть выбор чистить почты от кирилллицы удалением букв или перевод в статус "UNKNOWN".  
Но сейчас рассвет почт на кириллице, поэтому чистку применяю к почтам(доменам второго уровня),  
у которых запрещено использования кириллицы в логине(т.е. кириллица не используется).  
Запрос делал к каждому Российскому поставщику на начало 2022 года.

In [2]:
# Функция для проверки ошибок в личных почтах

def email_anomalies(df,columns):
    # Кол-во "@" знаков > 1
    several_dogs = df.select(columns).filter(col(columns).rlike("[@]{1,}$")).count()
    # Кол-во пробелов в начале
    start_clean = df.select(columns).filter(col(columns).rlike("^[ ]")).count()
    # Кол-во до двух знаков перед знаком "@"
    before_dot = df.select(columns).filter(col(columns).rlike("^[\w|\W]{0,2}@")).count()
    # Кол-во уникальных до двух знаков перед знаком "@"
    before_dot_distinct = df.select(columns).filter(col(columns).rlike("^[\w|\W]{0,2}@")).distinct().count()
    # Кол-во кириллицы до знаком "@"
    kirillica = df.select(columns).filter(col(columns).rlike("[А-Яа-я]@")).count()
    # Кол-во уникальных кириллицы до знаком "@"
    kirillica_distinct = df.select(columns).filter(col(columns).rlike("[А-Яа-я]@")).distinct().count()
    # Кол-во не допутимых с доменов mail.ru
    yandex = df.select(columns).filter(col(columns).rlike("(?i)[^A-Za-z][А-Яа-я]@yandex.ru|(?i)[^A-Za-z][А-Яа-я]@ya.ru")).count()
    # Кол-во уникальных не допутимых с доменов mail.ru
    yandex_unic = df.select(columns).filter(col(columns).rlike("(?i)[^A-Za-z][А-Яа-я]@yandex.ru|(?i)[^A-Za-z][А-Яа-я]@ya.ru")).distinct().count()
    # Кол-во не допутимых с доменов mail.ru
    mail = df.select(columns).filter(col(columns).rlike("(?i)[^A-Za-z][А-Яа-я]@mail.ru")).count()
    # Кол-во уникальных не допутимых с доменов mail.ru
    mail_unic = df.select(columns).filter(col(columns).rlike("(?i)[^A-Za-z][А-Яа-я]@mail.ru")).distinct().count()
    # Кол-во не допутимых с доменов gmail.ru
    gmail = df.select(columns).filter(col(columns).rlike("(?i)[^A-Za-z][А-Яа-я]@gmail.com")).count()
    # Кол-во уникальных не допутимых с доменов gmail.ru
    gmail_unic = df.select(columns).filter(col(columns).rlike("(?i)[^A-Za-z][А-Яа-я]@gmail.com")).distinct().count()

    print(f'Кол-во значений: {df.select(columns).count()}')
    print(f'Кол-во значений NaN : {df.select(columns).filter(col(columns).rlike("^[nan]*$|^[Nan]*$")).count()}')
    print(f'Кол-во "@" знаков > 1 : {several_dogs}')
    print(f'Кол-во пробелов в начале : {start_clean}')
    print(f'Кол-во уникальных почт без знака "@" : {df.select(columns).filter(col(columns).rlike("^[^@]*$")).distinct().count()}')
    print(f'Кол-во до двух знаков(символов) перед знаком "@" : {before_dot}')
    print(f'Кол-во уникальных до двух знаков перед знаком "@" : {before_dot_distinct}')
    print(f'Кол-во кириллицы до знака "@" : {kirillica}')
    print(f'Кол-во уникальных кириллицы до знака "@" : {kirillica_distinct}')
    print(f'Кол-во не допустимых с доменом yandex.ru : {yandex}')
    print(f'Кол-во уникальных не допустимых с доменом yandex.ru : {yandex_unic}')
    print(f'Кол-во не допустимых с доменом mail.ru : {mail}')
    print(f'Кол-во уникальных не допустимых с доменом mail.ru : {mail_unic}')
    print(f'Кол-во не допустимых с доменом gmail.ru : {gmail}')
    print(f'Кол-во уникальных не допустимых с доменом gmail.ru : {gmail_unic}')
    print(f'Странный пробел который не пробел : {df.select(column).filter(col(column).rlike(" ")).count()}')
    
email_anomalies(df_spark,col_15)

In [ ]:
# Проверка сколько почт являются забанненые

def check_domen(df,column,df1):
    df = df.withColumn('domen',split(F.col(column), '@')[1])
    df_domen = df.join(df1, df.domen==df1.domens_1,'left')\
        .select('domens_1').filter(col('domens_1').isNotNull()).count()
    
    return df_domen

check_domen(df_spark,col_15,df_domens)

In [ ]:
# Функция для очистки данных личных почт

def clean_email(df,column,df1):
    df_email = df.withColumn(column, F.regexp_replace(column, ' ',' '))\
        .withColumn(column, F.when(col(column).rlike("^[^@]*$"), 'UNKNOWN').otherwise(col(column)))\
        .withColumn(column, F.regexp_replace(column, ' |¶|\*|#|\|',''))\
        .withColumn(column, F.when(col(column).rlike("@[^.]+$"), 'UNKNOWN').otherwise(col(column)))\
        .withColumn(column, F.when(col(column).rlike("[@]{2,}$"), 'UNKNOWN').otherwise(col(column)))\
        .withColumn(column, F.when(col(column).rlike("[@]+[А-Яа-яЁёA-Za-z0-9_,.; :<>]+[@]+"), 'UNKNOWN').otherwise(col(column)))\
        .withColumn(column, F.regexp_replace(column, '^[!@#$%^&*()+=;:,/?\|\` ~\[\]{}^\'\"<>]+',''))\
        .withColumn(column, F.regexp_replace(column, '[!@#$%^&*()+=;:,/?\|\` ~\[\]{}^\'\"<>]+$',''))\
        .withColumn(column, F.regexp_replace(column, '(?i)[.]r$|(?i)[.]ry$','.ru'))\
        .withColumn(column, F.regexp_replace(column, '(?i)маil.ru$','mail.ru'))\
        .withColumn(column, F.regexp_replace(column, r'(?i)[А-Яа-яЁё]+(?=@myrambler.ru)|(?i)[А-Яа-яЁё]+(?=@autorambler.ru)|(?i)[А-Яа-яЁё]+(?=@lenta.ru)|(?i)[А-Яа-яЁё]+(?=@ro.ru)|(?i)[А-Яа-яЁё]+(?=@rambler.ua)|(?i)[А-Яа-яЁё]+(?=@rambler.ru)',''))\
        .withColumn(column, F.regexp_replace(column, r'(?i)[А-Яа-яЁё]+(?=@mail.ru)|(?i)[А-Яа-яЁё]+(?=@list.ru)|(?i)[А-Яа-яЁё]+(?=@bk.ru)|(?i)[А-Яа-яЁё]+(?=@internet.ru)|(?i)[А-Яа-яЁё]+(?=@inbox.ru)',''))\
        .withColumn(column, F.regexp_replace(column, r'(?i)[А-Яа-яЁё]+(?=@gmail.com)|(?i)[А-Яа-яЁё]+(?=@googlemail.com)',''))\
        .withColumn(column, F.regexp_replace(column, r'(?i)[А-Яа-яЁё]+(?=@yandex.ru)|(?i)[А-Яа-яЁё]+(?=@yandex.by)|(?i)[А-Яа-яЁё]+(?=@yandex.ua)|(?i)[А-Яа-яЁё]+(?=@yandex.kz)|(?i)[А-Яа-яЁё]+(?=@yandex.com)|(?i)[А-Яа-яЁё]+(?=@ya.ru)|(?i)[А-Яа-яЁё]+(?=@narod.ru)',''))\
        .withColumn(column, F.regexp_replace(column, r'(?i)[А-Яа-яЁё]+(?=@ymail.com)|(?i)[А-Яа-яЁё]+(?=@rocketmail.com)|(?i)[А-Яа-яЁё]+(?=@yahoo.com)',''))\
        .withColumn(column, F.regexp_replace(column, r'(?i)[А-Яа-яЁё]+(?=@hotmail.com)|(?i)[А-Яа-яЁё]+(?=@live.com)|(?i)[А-Яа-яЁё]+(?=@outlook.com)',''))\
        .withColumn(column, F.regexp_replace(column, r'(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@myrambler.ru)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@autorambler.ru)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@lenta.ru)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@ro.ru)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@rambler.ua)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@rambler.ru)',''))\
        .withColumn(column, F.regexp_replace(column, r'(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@mail.ru)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+list.ru)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@bk.ru)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+internet.ru)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+inbox.ru)',''))\
        .withColumn(column, F.regexp_replace(column, r'(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@gmail.com)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@googlemail.com)',''))\
        .withColumn(column, F.regexp_replace(column, r'(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@yandex.ru)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@yandex.by)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@yandex.ua)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@yandex.kz)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@yandex.com)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@ya.ru)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@narod.ru)',''))\
        .withColumn(column, F.regexp_replace(column, r'(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@ymail.com)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@rocketmail.com)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@yahoo.com)',''))\
        .withColumn(column, F.regexp_replace(column, r'(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@hotmail.com)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@live.com)|(?i)[А-Яа-яЁё< ]+(?=[A-Za-z0-9.]+@outlook.com)',''))\
        .withColumn(column, F.regexp_replace(column, '^[!@#$%^&*()+=;:,/?\|\` ~\[\]{}^\'\"<>]+',''))\
        .withColumn(column, F.regexp_replace(column, '[!@#$%^&*()+=;:,/?\|\` ~\[\]{}^\'\"<>]+$',''))\
        .withColumn(column, F.when(col(column).rlike("[.]\w$|[.]\W$"), 'UNKNOWN').otherwise(col(column)))\
        .withColumn(column, F.when(col(column).rlike("(?i)^[0-9][A-Za-zА-Яа-яЁёЁё0-9]+@yandex.by|(?i)^[0-9][A-Za-zА-Яа-яЁёЁё0-9]+@yandex.ua|(?i)^[0-9][A-Za-zА-Яа-яЁёЁё0-9]+@yandex.kz|(?i)^[0-9][A-Za-zА-Яа-яЁёЁё0-9]+@yandex.ru|(?i)^[0-9][A-Za-zА-Яа-яЁёЁё0-9]+@yandex.com|(?i)^[0-9][A-Za-zА-Яа-яЁёЁё0-9]+@ya.ru"), 'UNKNOWN').otherwise(col(column)))\
        .withColumn(column, F.when(col(column).rlike("^\w{0,2}@|^\W{0,2}@|^\w\W{0,2}@|^\W\w{0,2}@"), 'UNKNOWN').otherwise(col(column)))\
        .withColumn(column, F.regexp_replace(column, 'xn----7sba5bazifhfz.xn--p1ai$','спарта-кфкс.рф'))\
    
    df_email = df_email.withColumn('domen',split(F.col(column), '@')[1])\
        .withColumn('domen', F.regexp_replace('domen', '^[¶!@#$%^&*()+=;:,/?\|\`~\[\]{}^\'\"<>]',''))
    
    df_email = df_email.join(df1, df_email.domen==df1.domens_1,'left')\
        .withColumn(column, F.when(col('domens_1').isNotNull(), 'ban_SUDIR').otherwise(col(column)))\
        .drop('domen', 'domens_1')
#        .select(column).filter(col(column).rlike('[<>]')).show(100,False,True)
     
    return df_email

clean_email(df_spark,col_15,df_domens)

In [ ]:
# Описание функции для очистки данных личных почт

def clean_email(df,column,df1):
    # Удаляем пробел который видится как знак(два слова читаются как один)
    .withColumn(column, F.regexp_replace(column, ' ',' '))\
    # Убираем почты без точек после "@"
    .withColumn(column, F.when(col(column).rlike("@[^.]+$"), 'UNKNOWN').otherwise(col(column)))\
    # Убираем почты без "@"
    df_email = df.withColumn(column, F.when(col(column).rlike("^[^@]*$"), 'UNKNOWN').otherwise(col(column)))\
    # Убираем все пробелы 
    .withColumn(column, F.regexp_replace(column, ' ',''))\
    # Убираем если больше одной "@"
    .withColumn(column, F.when(col(column).rlike("[@]{1,}$"), 'UNKNOWN').otherwise(col(column)))\
    # Убираем "nan"
    .withColumn(column, F.when(col(column).rlike("^[nan]*$|^[Nan]*$"), 'UNKNOWN').otherwise(col(column)))\
    # Убираем до двух знаков и символов до "@"
    .withColumn(column, F.when(col(column).rlike("^\w{0,2}@|^\W{0,2}@|^\w\W{0,2}@|^\W\w{0,2}@"), 'UNKNOWN').otherwise(col(column)))\
    # Убираем символы в конце почты(точки и запятые)
    .withColumn(column, F.regexp_replace(column, '[¶.-><_]+$',''))\
    # Добавляем к "r" + "u" = ru
    .withColumn(column, F.regexp_replace(column, '[.]r$','.ru'))\
    # Убираем домен верхнего уровня с одним символом или знаком 
    .withColumn(column, F.when(col(column).rlike("[.]\w$|[.]\W$"), 'UNKNOWN').otherwise(col(column)))\
    # Заменяем Punycode на его расшифровку
    .withColumn('Email', F.regexp_replace('Email', 'xn----7sba5bazifhfz.xn--p1ai$','спарта-кфкс.рф'))\
    # Заменяем маil.ru(первая буква на русском) на mail.ru
    .withColumn(column, F.regexp_replace(column, '(?i)маil.ru$','mail.ru'))\
    # создаем колонку только домен второго уровня для проверки в забанненых 
    df_email = df_email.withColumn('domen',split(F.col(column), '@')[1])
    # Джойним список с доменами второго уровня которые забаннены
    df_email = df_email.join(df1, df_email.domen==df1.domens_1,'left')\
    # Убираем символы в конце почты(точки и запятые)
    .withColumn('domen', F.regexp_replace('domen', '^[.;,/!:#-]',''))\
    # В колонке 'Email' убираем почты если они есть в колонке забанных
    .withColumn(column, F.when(col('domens_1').isNotNull(), 'ban_SUDIR').otherwise(col(column)))\
    # Удаляем колонки которые добавили в текущей функции
    .drop('domen', 'domens_1')

    return df_email
clean_email(df_spark,col_15,df_domens)